    MongoDB trae en su manual oficial un dataset de restaurante para que juegues con el. Asi pues, tras descargarlo e importarlo a la mi servidor local. Veamos que tiene y que podemos sacar.
    
    Puedes encontrar el datase en: https://raw.githubusercontent.com/mongodb/docs-assets/primer-dataset/primer-dataset.json

In [3]:
import pandas as pd
from pymongo import MongoClient

In [4]:
client = MongoClient()

Lo primero sera montar nuestra conexion con la base de datos para explorar que colecciones tiene y cuantos documentos tiene cada coleccion.

In [5]:
#client.database_names() nos permite ver que base de datos tenemos dentro del servidor mongo
#dbc.command('collstats','restaurants') nos muestra toda la informacion relacionada con una coleccion, pero tengo que
#investigarla aún mas para poder usarla adecuadamente.

In [6]:
dbc = client.test  
dbc.collection_names()

['restaurants']

In [7]:
test = dbc.restaurants
test.count()

25359

Solo una coleccion, sin embargo tiene más de 25 mil documentos lo cual lo convierte en un buen candidato para cargar todos los datos. Por lo que el siguiente paso seria ver como suelen ser los documentos, para ello cargaremos uno y cual es su estrutura.

Nota: Dado que es mongoDB, en una coleccion cada estrutura puede variar. Es decir, puede haber documentos que tenga atributos que no otros no, o le asginen valores de distinto tipo al mismo atributo. Este es solo orientativo.

In [8]:
test.find_one() #seleccionamos uno al azar para ver que estrutura suele tener

{'_id': ObjectId('5a69273157f6a3e3520e8426'),
 'address': {'building': '2780',
  'coord': [-73.98241999999999, 40.579505],
  'street': 'Stillwell Avenue',
  'zipcode': '11224'},
 'borough': 'Brooklyn',
 'cuisine': 'American',
 'grades': [{'date': datetime.datetime(2014, 6, 10, 0, 0),
   'grade': 'A',
   'score': 5},
  {'date': datetime.datetime(2013, 6, 5, 0, 0), 'grade': 'A', 'score': 7},
  {'date': datetime.datetime(2012, 4, 13, 0, 0), 'grade': 'A', 'score': 12},
  {'date': datetime.datetime(2011, 10, 12, 0, 0), 'grade': 'A', 'score': 12}],
 'name': 'Riviera Caterer',
 'restaurant_id': '40356018'}

    Las querys de mongo, devuelven un cursor, pero estos cursores se pueden convertir en listas o al estar en formato bson, son facilmente convertible a json. Sin embargo para este caso, nos decantaremos por convertir los datos a una lista y tranformarlo en un dataset.

In [9]:
type(test.find())

pymongo.cursor.Cursor

In [10]:
lista = list(test.find({'borough':'Brooklyn'}))
lista[0]

{'_id': ObjectId('5a69273157f6a3e3520e8426'),
 'address': {'building': '2780',
  'coord': [-73.98241999999999, 40.579505],
  'street': 'Stillwell Avenue',
  'zipcode': '11224'},
 'borough': 'Brooklyn',
 'cuisine': 'American',
 'grades': [{'date': datetime.datetime(2014, 6, 10, 0, 0),
   'grade': 'A',
   'score': 5},
  {'date': datetime.datetime(2013, 6, 5, 0, 0), 'grade': 'A', 'score': 7},
  {'date': datetime.datetime(2012, 4, 13, 0, 0), 'grade': 'A', 'score': 12},
  {'date': datetime.datetime(2011, 10, 12, 0, 0), 'grade': 'A', 'score': 12}],
 'name': 'Riviera Caterer',
 'restaurant_id': '40356018'}

In [11]:
df = pd.DataFrame(lista)
df.head()

,_id,address,borough,cuisine,grades,name,restaurant_id
0,5a69273157f6a3e3520e8426,"{'building': '2780', 'coord': [-73.98241999999...",Brooklyn,American,"[{'score': 5, 'grade': 'A', 'date': 2014-06-10...",Riviera Caterer,40356018
1,5a69273157f6a3e3520e8428,"{'building': '469', 'coord': [-73.961704, 40.6...",Brooklyn,Hamburgers,"[{'score': 8, 'grade': 'A', 'date': 2014-12-30...",Wendy'S,30112340
2,5a69273157f6a3e3520e842d,"{'building': '7114', 'coord': [-73.9068506, 40...",Brooklyn,Delicatessen,"[{'score': 10, 'grade': 'A', 'date': 2014-05-2...",Wilken'S Fine Food,40356483
3,5a69273157f6a3e3520e842e,"{'building': '6409', 'coord': [-74.00528899999...",Brooklyn,American,"[{'score': 12, 'grade': 'A', 'date': 2014-07-1...",Regina Caterers,40356649
4,5a69273157f6a3e3520e8430,"{'building': '1269', 'coord': [-73.871194, 40....",Brooklyn,Chinese,"[{'score': 21, 'grade': 'B', 'date': 2014-09-1...",May May Kitchen,40358429


Vaya, algo extraño sucede con nuestro nuevo dataset. Parace ser que los atributos combuesto por varios atributos son interpretado como un unico valor, lo cual afea y hace más dificil su analisis. Para solucionar este problema deberemos cavar en cada columna.

In [60]:
df['coords'] = df['address'].apply(lambda x : x['coord'])
df['address'] = df ['address'].apply(lambda x : x.values())
df['address'][0]


dict_values(['2780', [-73.98241999999999, 40.579505], 'Stillwell Avenue', '11224'])

# LISTA DE PROBLEMAS A SOLUCIONAR:
No se han comprobado de forma adecuada la estrutura general y las excepciones de los documentos. Lo cual va a dar problemas.